In [1]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def get_soup_content(url):
    resp=r.get(url).content
    BeautifulSoup(resp)
    return soup   

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

def get_soup_from_selenium(url):
    driver= webdriver.Chrome('/Users/apple/Desktop/chromedriver')
    driver.get(url)
    delay = 50 # seconds
    try:
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, 'search_location_state')))
        print ("Page is ready!")
        soup = BeautifulSoup(driver.page_source,"lxml")
    except TimeoutException:
        print ("Loading took too much time!")
    return soup
    
get_soup_from_selenium('https://www.startupranking.com/top/0/2')

Page is ready!


<html lang="en-US" xmlns:fb="https://www.facebook.com/2008/fbml"><head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/><script async="" crossorigin="anonymous" src="https://connect.facebook.net/en_US/sdk.js?hash=98846a04a60623e64dbcd328387bd27e&amp;ua=modern_es6"></script><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script id="twitter-wjs" src="https://platform.twitter.com/widgets.js"></script><script id="facebook-jssdk" src="//connect.facebook.net/en_US/sdk.js#version=v2.0&amp;xfbml=1&amp;appId=373997469370914"></script><script async="" charset="utf-8" src="//v2.zopim.com/?26WA6gMraAVJgC9Wd6lZjX8OddzMtb6Y" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-5Q72BHH"></script><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"128447d4e4",applicationID:"19195483"};window.NREUM||(NREUM={}),__nr_require=function

In [4]:
def scrap_home_page(url):
    print(url)
    soup=get_soup_from_selenium(url)
    tabs=soup.find_all('table',{'class':'rank_table'})[0]
    rows=tabs.find_all('tr')
    list_urls=[i.select('a')[0].get('href') for i in rows]
    list_urls.pop(0)
    return list_urls

scrap_home_page('https://www.startupranking.com/top')

https://www.startupranking.com/top
Page is ready!


['/500px',
 '/canva',
 '/duolingo',
 '/coursera',
 '/giphy',
 '/freelancer',
 '/coinbase',
 '/telegram',
 '/buffer',
 '/ifttt',
 '/transferwise',
 '/teespring',
 '/doordash',
 '/ahrefs',
 '/fiverr',
 '/wechat',
 '/traveloka',
 '/skillshare',
 '/sproutsocial',
 '/wetransfer',
 '/zapier',
 '/about-me',
 '/trendyol',
 '/tradingview',
 '/designmodo',
 '/pearltrees',
 '/instacart',
 '/bukalapak',
 '/revolut',
 '/8tracks',
 '/teachable',
 '/sticker-mule',
 '/babbel',
 '/treehouse',
 '/stores',
 '/mapbox',
 '/localbitcoins',
 '/skoob',
 '/base',
 '/spreaker',
 '/letterboxd',
 '/strava',
 '/typeform',
 '/toggl',
 '/piktochart',
 '/picsart',
 '/konga',
 '/gleam',
 '/hackerrank',
 '/blibli',
 '/jusbrasil',
 '/freshdesk',
 '/hackerearth',
 '/smallpdf',
 '/stripe',
 '/ziprecruiter',
 '/enjoei',
 '/camp-fire-jp',
 '/datacamp',
 '/jumia-eg',
 '/twilio',
 '/bigfish',
 '/qiita',
 '/unbounce',
 '/pcloud',
 '/ecosia',
 '/taboola',
 '/bankbazaar-com',
 '/sellfy',
 '/retty',
 '/dashlane',
 '/thinkific',
 

In [ ]:
def scrap_startup_page(url_startup,total_funding_amount):

    resp2=r.get(url_startup).content
    
    soup2=BeautifulSoup(resp2)
    country_name = soup2.select('img.flag')[1].get('alt')
    name=soup2.select('div.su-info h2 a')[0].text
    # founded date
    tabb = soup2.select('div.su-info p.su-loc')
    if len(tabb)==0: 
        founded_date = ''
    else:
        founded_date =  tabb[0].text.strip()
    
    cat = soup2.select('div.su-tags.group ul li')
    category_list =[i.text for i in cat]
    category_string = ','.join(category_list)
    dict_info={'name':[name],'country_name':[country_name],'founded_date':[founded_date],'category_list': [category_string],'total_funding_amout':[total_funding_amount]}
    df=pd.DataFrame(dict_info)
    return df



scrap_startup_page('https://www.startupranking.com/ifttt',210100000)

In [8]:
def get_url_funding_page(url_startup_page):
    resp=r.get(url_startup_page).content
    soup=BeautifulSoup(resp)
    tabb = soup.select('.small-table-container')[1].select('a.see-all')
    if len(tabb)==0: 
        return ''
    else:
        return tabb[0].get('href')
get_url_funding_page('https://www.startupranking.com/ifttt')

'/startup/ifttt/funding-rounds'

In [9]:
import re
def get_total_funding_amount(url_funding_page):
    if url_funding_page=='':
        return 0
    else:
        resp=r.get(url_funding_page).content
        soup=BeautifulSoup(resp)
        tabs2 = soup.select('.ranks')[0]
        rows2=tabs2.find_all('tr')
        list_funding_amount = [i.select('span')[0].text for i in rows2]
        list_funding_amount = list(filter(lambda a: a != 'Undisclosed amount', list_funding_amount))
        funding_amount_dolar = [int(re.search(r'[0-9]+',i.replace(',','')).group(0)) for i in list_funding_amount]
        Total_funding_amount = sum(funding_amount_dolar)
        return Total_funding_amount

get_total_funding_amount('https://www.startupranking.com/startup/ifttt/funding-rounds')

38500000

In [10]:
import pymysql
data = scrap_startup_page('https://www.startupranking.com/500px',0,210100000)
def save_to_sql(data):
    connection = pymysql.connect(host='localhost',
                             user='root',
                             password='safa',
                             db='startupdb')
    cursor = connection.cursor()
    # creating column list for insertion
    cols = "`,`".join([str(i) for i in data.columns.tolist()])

    # Insert DataFrame recrds one by one.
    for i,row in data.iterrows():
        sql = "INSERT INTO `startup` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()
save_to_sql(data)

In [11]:
import time


def scrap_all_startup_perpage(url_home,list_urls):
    dataframe = pd.DataFrame()
    i=0
    for url_startup in list_urls:
        i+=1
        whole_url_startup = url_home + url_startup
        print('url =', whole_url_startup)
        
        url_funding = get_url_funding_page(whole_url_startup)
        if url_funding=='':
            whole_url_funding = ''
        else:
            whole_url_funding = url_home + url_funding
       # dataframe['total_funding_amout'] = get_total_funding_amount(whole_url_funding)
        total = get_total_funding_amount(whole_url_funding)
        new_df = scrap_startup_page(whole_url_startup,i,total)
        dataframe = dataframe.append(new_df)
        time.sleep(1)
        if i%10==0:
            pos = i-10
            print(f'dataframe.iloc[{pos},{i}]')
            df = dataframe.iloc[pos:i]
            save_to_sql(df)
    
    return dataframe

url = 'https://www.startupranking.com/top'
url_home = 'https://www.startupranking.com'
list_urls = scrap_home_page(url)
dataframe = scrap_all_startup_perpage(url_home,list_urls)

https://www.startupranking.com/top
Page is ready!
url = https://www.startupranking.com/500px
url = https://www.startupranking.com/canva
url = https://www.startupranking.com/duolingo
url = https://www.startupranking.com/coursera
url = https://www.startupranking.com/giphy
url = https://www.startupranking.com/freelancer
url = https://www.startupranking.com/coinbase
url = https://www.startupranking.com/telegram
url = https://www.startupranking.com/buffer
url = https://www.startupranking.com/transferwise
dataframe.iloc[0,10]
url = https://www.startupranking.com/ifttt
url = https://www.startupranking.com/teespring
url = https://www.startupranking.com/doordash
url = https://www.startupranking.com/ahrefs
url = https://www.startupranking.com/fiverr
url = https://www.startupranking.com/wechat
url = https://www.startupranking.com/traveloka
url = https://www.startupranking.com/skillshare
url = https://www.startupranking.com/sproutsocial
url = https://www.startupranking.com/wetransfer
dataframe.ilo

In [12]:
dataframe

,name,description,founded_date,category_list,total_funding_amout
1,500px,We're the premier community for inspiring phot...,"Founded: October 10, 2009","App,Mobile,Photographers,Photos,Web,Photograph...",22325000
2,Canva,Amazingly simple graphic design. Canva makes g...,"Founded: January 01, 2012","Web Design,Simple,Graphic Design,Design,Tool,P...",49555000
3,Duolingo,Free language education for the world. Accordi...,"Founded: November 01, 2011","E-Learning,Education,Internet,Language Learning",108300000
4,Coursera,Coursera - We are a social entrepreneurship co...,"Founded: January 01, 2012","Edtech,Education,Enterprise,Internet",210100000
5,Giphy,Animated GIF search. The first and largest GIF...,"Founded: February 01, 2013","Internet,Photo Sharing,Search Engine",150949998
...,...,...,...,...,...
96,Contentful,Contentful is a flexible and future-friendly c...,"Founded: June 26, 2013","Apps,Content,Mobile,Web Cms,CMS,Cloud Computin...",16800000
97,PakWheels.com,PakWheels.com is an online community of automo...,"Founded: July 18, 2003","Automobile,Community,Social,Web,Automotive,Aut...",0
98,Lyst,Lyst is a personalized fashion marketplace tha...,"Founded: January 01, 2010","Customized Merchandise,Marketplaces,Shopping,E...",59950000
99,Plotly,"Plotly - It is a free, online, graphing, curve...","Founded: August 09, 2013","Online,Free,Infographic,Analytics,Big Data,Col...",10113948


In [24]:
def iterate_startup_page(n,url_home,url_top):
    dataframe = pd.DataFrame()
    for i in range(int(n)):
        url_new = url_top+'/'+str(i+1)
        list_urls = scrap_home_page(url_new)
        df = scrap_all_startup_perpage(url_home,list_urls)
        dataframe = dataframe.append(df)     
    dataframe.to_csv('startup2.csv')
    return dataframe

dataframe2 = iterate_startup_page('2','https://www.startupranking.com','https://www.startupranking.com/top/0')

https://www.startupranking.com/top/0/1
Page is ready!
url = https://www.startupranking.com/500px
url = https://www.startupranking.com/canva
url = https://www.startupranking.com/duolingo
url = https://www.startupranking.com/coursera
url = https://www.startupranking.com/giphy
url = https://www.startupranking.com/freelancer
url = https://www.startupranking.com/coinbase
url = https://www.startupranking.com/telegram
url = https://www.startupranking.com/buffer
url = https://www.startupranking.com/transferwise
dataframe.iloc[0,10]
url = https://www.startupranking.com/ifttt
url = https://www.startupranking.com/teespring
url = https://www.startupranking.com/doordash
url = https://www.startupranking.com/ahrefs
url = https://www.startupranking.com/fiverr
url = https://www.startupranking.com/wechat
url = https://www.startupranking.com/traveloka
url = https://www.startupranking.com/skillshare
url = https://www.startupranking.com/sproutsocial
url = https://www.startupranking.com/wetransfer
dataframe

url = https://www.startupranking.com/pixatool
url = https://www.startupranking.com/practo
dataframe.iloc[60,70]
url = https://www.startupranking.com/urban-ladder
url = https://www.startupranking.com/wantedly
url = https://www.startupranking.com/iubenda
url = https://www.startupranking.com/mailerlite
url = https://www.startupranking.com/aglasem
url = https://www.startupranking.com/buyma
url = https://www.startupranking.com/gilabola-com
url = https://www.startupranking.com/leetchi-com
url = https://www.startupranking.com/bebee
url = https://www.startupranking.com/cacoo
dataframe.iloc[70,80]
url = https://www.startupranking.com/conduit
url = https://www.startupranking.com/passei-direto
url = https://www.startupranking.com/tailor-brands
url = https://www.startupranking.com/crowdworks
url = https://www.startupranking.com/thrillophilia
url = https://www.startupranking.com/baomoi
url = https://www.startupranking.com/qobuz
url = https://www.startupranking.com/traveltriangle
url = https://www.s

In [25]:
dataframe2

,name,description,founded_date,category_list,total_funding_amout
0,500px,We're the premier community for inspiring phot...,"Founded: October 10, 2009","App,Mobile,Photographers,Photos,Web,Photograph...",22325000
0,Canva,Amazingly simple graphic design. Canva makes g...,"Founded: January 01, 2012","Web Design,Simple,Graphic Design,Design,Tool,P...",49555000
0,Duolingo,Free language education for the world. Accordi...,"Founded: November 01, 2011","E-Learning,Education,Internet,Language Learning",108300000
0,Coursera,Coursera - We are a social entrepreneurship co...,"Founded: January 01, 2012","Edtech,Education,Enterprise,Internet",210100000
0,Giphy,Animated GIF search. The first and largest GIF...,"Founded: February 01, 2013","Internet,Photo Sharing,Search Engine",150949998
...,...,...,...,...,...
0,Workana,Workana - We help our customers to find the be...,"Founded: January 01, 2012","Jobs,Recruitment,Talent,E-Commerce,Outsourcing",5100000
0,Bitso,"Bitso is Mexico's leading Bitcoin, Ether, and ...","Founded: April 01, 2014","Bitcoin,Crypto Currency,Bitcoins,Digital Curre...",4350000
0,RareJob.com,RareJob - Online English Tutorial. RareJob is ...,,"Chat,Language Learning,English Speaking World",0
0,Kanui,Kanui is a Brazilian e-commerce focused on fas...,"Founded: January 01, 2011","E-Commerce,Fashion,Marketplaces,Sports",0


In [ ]:
if __name__=='__main__':
    dataframe = pd.DataFrame()
    list_urls = scrap_home_page(url_new)